In [1]:
import numpy as np
from utils.ifa_meander_project.meander_function import *

In [2]:
mesh_name = "ifa_meander_2.msh"
save_mesh_folder = 'data/gmsh_files/'
ifa_meander_mat = 'data/antennas_mesh/ifa_meander_2.mat'
ifa_meander_msh = save_mesh_folder + mesh_name

In [3]:
fC      = 868e6
fLow    = 820e6
fHigh   = 940e6
nPoints = calculate_nPoints(fLow, fHigh, fC)
accuracy= 1e6
print(f"nPoints = {nPoints}")

nPoints = 11


In [4]:
vitesse_lumiere = 3.0e8  # en mètres par seconde (m/s)
longueur_onde = vitesse_lumiere / fC 
quart_d_onde = longueur_onde / 2

print(f"Pour la fréquence de raisonnance {fC} Hz :")
print(f"Longueur d'onde (λ) = {longueur_onde} m")
print(f"Quart d'onde (λ/4) = {quart_d_onde} m")

Pour la fréquence de raisonnance 868000000.0 Hz :
Longueur d'onde (λ) = 0.3456221198156682 m
Quart d'onde (λ/4) = 0.1728110599078341 m


frequencies = np.linspace(fLow, fHigh, nPoints)
print("Liste des frequences : ")
count = 1
for frequency in frequencies:
    print(f"Simulation Numéro {count} = {frequency / 1e6:.2f} MHz")
    count += 1

In [5]:
# Terminal size 
t_a = 50 / 1000   # Hauteur
t_b = 30 / 1000  # Largeur
x_t = np.array([-t_a, 0, 0, -t_a])
y_t = np.array([t_b/2, t_b/2, -t_b/2, -t_b/2])

In [6]:
# Génération des points du contour IFA
# wid has to be less than min_slot 
a = 15 / 1000    
b = 30 / 1000       
wid = 0.5 / 1000    # Largeur des branches
min_slot = 3 / 1000
Lenght_antenna = quart_d_onde

# feed_wid = 1 / 1000
feed_wid = wid
# feed_point = [0, t_b/2 - wid/2, 0]
feed = t_b/2 - 4.28 / 1000
feed_point = [0, feed, 0]

In [ ]:
redimensionner = True
while redimensionner:
    x, y, longueur_meandre = ifa_creation_center(Lenght_antenna, a, b, wid, min_slot)
    if longueur_meandre < Lenght_antenna:
        min_slot -= 0.5 / 1000
    else:
        redimensionner = False

# print(f"Longueur Meandre = {longueur_meandre}")
x_m, y_m = trace_meander(x, y, wid)
feed_x = np.array([0, min_slot-wid/2, min_slot-wid/2,0])
feed_y = np.array([feed_wid/2, feed_wid/2, -feed_wid/2, -feed_wid/2])
antenna_ifa_meander(x_m, y_m, x_t, y_t, feed_x, feed_y, save_mesh_folder, mesh_name, 2.25/1000)

longueur_meandre = 0.133 

longueur_desiree = 0.1728110599078341
Longueur Meandre = 0.133
The .msh file was successfully saved to: 'data/gmsh_files/ifa_meander_2.msh'


In [ ]:
extract_msh_to_mat(ifa_meander_msh, ifa_meander_mat)
s11_db, f_resonance, has_converged = simulate_freq_loop(fLow, fHigh, nPoints, fC, accuracy, ifa_meander_mat, feed_point)
plot_s11_curve(fLow, fHigh, nPoints, s11_db, fC)

In [ ]:
has_converged = False
max_iterations = 5
iteration = 0

while not has_converged and iteration < max_iterations:
    print(f"wid = {wid}\n")
    print(f"min_slot = {min_slot}\n")
    x, y = ifa_creation_center(Lenght_antenna, a, b, wid, min_slot)
    x_m, y_m = trace_meander(x, y, wid)
    feed_x = np.array([0, min_slot-wid/2, min_slot-wid/2,0])
    feed_y = np.array([feed + feed_wid/2, feed + feed_wid/2, feed -feed_wid/2, feed -feed_wid/2])
    antenna_ifa_meander(x_m, y_m, x_t, y_t, feed_x, feed_y, save_mesh_folder, mesh_name, 2.25/1000)
    extract_msh_to_mat(ifa_meander_msh, ifa_meander_mat)
    s11_db, f_resonance, has_converged = simulate_freq_loop(fLow, fHigh, nPoints, fC, accuracy, ifa_meander_mat, feed_point)
    plot_s11_curve(fLow, fHigh, nPoints, s11_db, fC)
    iteration += 1

    if has_converged:
        print(f"Convergence atteinte à l'itération {iteration}")
    else:
        print("Convergence non atteinte")
        scale = f_resonance / fC
        print(f"Le Scale est : {scale}")
        # 5) Mise à jour des dimensions
        # wid      /= scale
        min_slot *= scale
        print(f"f_resonance / fC = {f_resonance / fC}")
        print("\n")


In [ ]:
scale = 2500 / 868
print(f"scale = {scale}")
min_slot = 3 / 1000
print(f"min_slot = {min_slot}")
min_slot *= scale
print(f"min_slot = {min_slot}")